In [15]:
import torch
import torchvision
from torch import nn
from inceptionV1 import InceptionV1

from torch.utils.data import DataLoader, random_split
# from torchvision import datasets
from torchvision import transforms

from tqdm import tqdm

In [2]:
inception = InceptionV1(n_classes = 1000, module_option = 'reduction')
inception

InceptionV1(
  (conv1): convBlock(
    (convBlock): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv2_1): convBlock(
    (convBlock): Sequential(
      (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
  (conv2_2): convBlock(
    (convBlock): Sequential(
      (0): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (inception3a): reducInception(
    (branch1): convBlock(
      (convBlock): S

# Output Shape Test

In [9]:
trans = transforms.Compose([transforms.Resize((224, 224)),
                            transforms.ToTensor(),
                            transforms.Normalize((0.485,0.456,0.406), (0.229, 0.224, 0.225))])

image_data = torchvision.datasets.ImageFolder(root = './data/Images', transform = trans)
train_size = int(image_data.__len__() * 0.8)
test_size = image_data.__len__() - train_size
train_data, test_data = random_split(image_data, [train_size, test_size])

output, aux1, aux2 = inception(train_data.__getitem__(0)[0].unsqueeze(0))
output.shape, aux1.shape, aux2.shape

(torch.Size([1, 1000]), torch.Size([1, 1000]), torch.Size([1, 1000]))

In [6]:
aux2

tensor([[-2.0758e-01,  2.3931e-01,  3.4748e-02,  2.9155e-01, -3.8265e-01,
         -2.1936e-01,  1.7093e-01, -2.5194e-02,  6.7638e-01, -7.1015e-02,
         -3.6385e-01,  1.8249e-01, -3.9608e-02, -2.3436e-01, -1.3549e-02,
         -3.7927e-01, -7.3572e-01, -2.8600e-01,  1.0674e-01,  1.3788e-01,
         -5.7804e-01, -3.1642e-02, -8.3274e-01,  1.4312e-01,  3.1461e-01,
         -1.4095e-01,  1.9127e-01, -2.9706e-01, -2.5110e-01,  7.0544e-02,
         -2.9801e-01,  8.1551e-02,  1.3998e-01,  4.1295e-01,  6.2329e-02,
         -5.9659e-01, -8.5580e-03,  1.3033e-01,  1.0476e-02, -5.4964e-02,
          2.3560e-01,  1.0165e-01,  5.2507e-01, -2.0342e-01,  2.8061e-01,
          9.2781e-01, -4.8023e-01,  8.1576e-02, -5.2840e-01, -5.2682e-01,
          7.8282e-01, -3.1535e-01,  4.0048e-01, -6.0265e-02,  2.3842e-01,
         -3.2917e-01, -1.6407e-01, -1.7929e-01, -3.3149e-02,  1.7055e-01,
          1.5953e-01, -9.3342e-03, -5.0386e-02, -2.0613e-01, -1.5977e-01,
         -6.5771e-02, -2.8726e-01,  3.

# Run

In [10]:
trans = transforms.Compose([transforms.Resize((224, 224)),
                            transforms.ToTensor(),
                            transforms.Normalize((0.485,0.456,0.406), (0.229, 0.224, 0.225))])
image_data = torchvision.datasets.ImageFolder(root = './data/Images', transform = trans)

In [11]:
train_size = int(image_data.__len__() * 0.8)
test_size = image_data.__len__() - train_size
train_data, test_data = random_split(image_data, [train_size, test_size])

In [12]:
train_dataloader = DataLoader(train_data, batch_size = 2, shuffle = True)
test_dataloader = DataLoader(test_data, batch_size = 2, shuffle = True)

In [13]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [16]:
inception = inception.to(device)

epochs = 1
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.Adam(inception.parameters(), lr = 1e-3)

In [ ]:
inception.train()
for epoch in tqdm(range(epochs)):
    running_loss = 0.0
    
    for i, data in enumerate(train_dataloader):
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs, aux1, aux2 = inception(images)
        real_loss = criterion(outputs, labels)
        aux1_loss = criterion(aux1, labels)
        aux2_loss = criterion(aux2, labels)
        
        loss = real_loss + 0.3 * aux1_loss + 0.3 * aux2_loss
        
        loss.backward()
        optimizer.step()
        
        if loss.item() > 1000:
            print(loss.item())
            for param in inception.parameters():
                print(param.data)
        
        running_loss += loss.item()
        if i % 100 == 0:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            print(f'Real Loss: {real_loss}, Aux1 Loss: {aux1_loss}, Aux2 Loss: {aux2_loss}')
            running_loss = 0.0
print('Training Finishied')

In [20]:
correct = 0
total = 0
# 학습 중이 아니므로, 출력에 대한 변화도를 계산할 필요가 없습니다
with torch.no_grad():
    for data in tqdm(test_dataloader):
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        # 신경망에 이미지를 통과시켜 출력을 계산합니다
        outputs, aux1, aux2 = inception(images)
        # 가장 높은 값(energy)를 갖는 분류(class)를 정답으로 선택하겠습니다
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct // total} %')

100%|██████████| 2058/2058 [01:30<00:00, 22.85it/s]

Accuracy of the network on the test images: 1 %


In [13]:
classes = image_data.classes

# 각 분류(class)에 대한 예측값 계산을 위해 준비
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# 변화도는 여전히 필요하지 않습니다
with torch.no_grad():
    for data in tqdm(test_dataloader):
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        
        outputs = inception(images)
        _, predictions = torch.max(outputs, 1)
        # 각 분류별로 올바른 예측 수를 모읍니다
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

100%|██████████| 2058/2058 [01:15<00:00, 27.24it/s]

Accuracy for class: n02085620-Chihuahua is 0.0 %
Accuracy for class: n02085782-Japanese_spaniel is 0.0 %
Accuracy for class: n02085936-Maltese_dog is 0.0 %
Accuracy for class: n02086079-Pekinese is 0.0 %
Accuracy for class: n02086240-Shih-Tzu is 0.0 %
Accuracy for class: n02086646-Blenheim_spaniel is 0.0 %
Accuracy for class: n02086910-papillon is 0.0 %
Accuracy for class: n02087046-toy_terrier is 100.0 %
Accuracy for class: n02087394-Rhodesian_ridgeback is 0.0 %
Accuracy for class: n02088094-Afghan_hound is 0.0 %
Accuracy for class: n02088238-basset is 0.0 %
Accuracy for class: n02088364-beagle is 0.0 %
Accuracy for class: n02088466-bloodhound is 0.0 %
Accuracy for class: n02088632-bluetick is 0.0 %
Accuracy for class: n02089078-black-and-tan_coonhound is 0.0 %
Accuracy for class: n02089867-Walker_hound is 0.0 %
Accuracy for class: n02089973-English_foxhound is 0.0 %
Accuracy for class: n02090379-redbone is 0.0 %
Accuracy for class: n02090622-borzoi is 0.0 %
Accuracy for class: n02090

In [14]:
# 각 분류별 정확도(accuracy)를 출력합니다
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: n02085620-Chihuahua is 0.0 %
Accuracy for class: n02085782-Japanese_spaniel is 0.0 %
Accuracy for class: n02085936-Maltese_dog is 0.0 %
Accuracy for class: n02086079-Pekinese is 0.0 %
Accuracy for class: n02086240-Shih-Tzu is 0.0 %
Accuracy for class: n02086646-Blenheim_spaniel is 0.0 %
Accuracy for class: n02086910-papillon is 0.0 %
Accuracy for class: n02087046-toy_terrier is 100.0 %
Accuracy for class: n02087394-Rhodesian_ridgeback is 0.0 %
Accuracy for class: n02088094-Afghan_hound is 0.0 %
Accuracy for class: n02088238-basset is 0.0 %
Accuracy for class: n02088364-beagle is 0.0 %
Accuracy for class: n02088466-bloodhound is 0.0 %
Accuracy for class: n02088632-bluetick is 0.0 %
Accuracy for class: n02089078-black-and-tan_coonhound is 0.0 %
Accuracy for class: n02089867-Walker_hound is 0.0 %
Accuracy for class: n02089973-English_foxhound is 0.0 %
Accuracy for class: n02090379-redbone is 0.0 %
Accuracy for class: n02090622-borzoi is 0.0 %
Accuracy for class: n02090